In [1]:
import os
import qiime2 as q2
from biom import Table
from skbio import TreeNode
from qiime2.plugins.umap.actions import pipeline, pipeline_phylogenetic
from qiime2.plugins.phylogeny.methods import midpoint_root
from qiime2.plugins.diversity.actions import core_metrics_phylogenetic



In [2]:
# tree 
tree_ = q2.Artifact.import_data('Phylogeny[Unrooted]', 'data/resources/97_otus.tree')
tree_rooted = midpoint_root(tree_)
# fix tree rooting issues
tree = tree_rooted.rooted_tree.view(TreeNode)
for n in tree.postorder(include_self=True):
    if n.length is None:
        n.length = 0
rooted_tree_fixed = q2.Artifact.import_data('Phylogeny[Rooted]',
                                            tree)
# save
rooted_tree_fixed.save('data/resources/97_otus.qza')

'data/resources/97_otus.qza'

In [4]:
# keyboard dataset
kb_table = q2.Artifact.import_data('FeatureTable[Frequency]',
                                   'data/keyboard/46809_otu_table.biom').view(Table)
kb_mf = q2.Metadata.load('data/keyboard/232_20170409-171325.txt').to_dataframe()
kb_mf = kb_mf[kb_mf.host_subject_id.isin(['M2','M3', 'M9'])]
kb_table = kb_table.filter(kb_mf.index)
kb_table = kb_table.filter(kb_table.ids('observation')[kb_table.sum('observation') > 0],
                           'observation')
kb_table = q2.Artifact.import_data('FeatureTable[Frequency]',
                                   kb_table)
kb_mf = q2.Metadata(kb_mf)

# 88 soils
soil_table = q2.Artifact.import_data('FeatureTable[Frequency]',
                                     'data/soils/44763_otu_table.biom').view(Table)
soil_mf = q2.Metadata.load('data/soils/103_20171215-103619.txt').to_dataframe()
# filter sample with 2 reads
soil_table = soil_table.filter(soil_table.ids('sample')[soil_table.sum('sample') > 500],
                               'sample')
soil_table = soil_table.filter(soil_table.ids('observation')[soil_table.sum('observation') > 0],
                               'observation')
soil_mf = q2.Metadata(soil_mf.reindex(soil_table.ids()))
soil_table = q2.Artifact.import_data('FeatureTable[Frequency]',
                                     soil_table)


In [3]:
kbue_res = pipeline(kb_table,
                    kb_mf,
                    umap_metric = 'euclidean')
kbur_res = pipeline(kb_table,
                    kb_mf,
                    sampling_depth = 500,
                    umap_metric = 'euclidean')
kbua_res = pipeline(kb_table,
                    kb_mf,
                    umap_metric = 'aitchison')
kbub_res = pipeline(kb_table,
                    kb_mf,
                    sampling_depth = 500,
                    umap_metric = 'braycurtis')
kbuj_res = pipeline(kb_table,
                    kb_mf,
                    sampling_depth = 500,
                    umap_metric = 'jaccard')
kbuu_res = pipeline_phylogenetic(kb_table,
                                 rooted_tree_fixed,
                                 kb_mf,
                                 sampling_depth = 500,
                                 umap_metric = 'unweighted_unifrac')
kbuw_res = pipeline_phylogenetic(kb_table,
                                 rooted_tree_fixed,
                                 kb_mf,
                                 sampling_depth = 500,
                                 umap_metric = 'weighted_unifrac')
slue_res = pipeline(soil_table,
                    soil_mf,
                    umap_metric = 'euclidean')
slur_res = pipeline(soil_table,
                    soil_mf,
                    sampling_depth = 500,
                    umap_metric = 'euclidean')
slua_res = pipeline(soil_table,
                    soil_mf,
                    umap_metric = 'aitchison')
slub_res = pipeline(soil_table,
                    soil_mf,
                    sampling_depth = 500,
                    umap_metric = 'braycurtis')
sluj_res = pipeline(soil_table,
                    soil_mf,
                    sampling_depth = 500,
                    umap_metric = 'jaccard')
sluu_res = pipeline_phylogenetic(soil_table,
                                 rooted_tree_fixed,
                                 soil_mf,
                                 sampling_depth = 500,
                                 umap_metric = 'unweighted_unifrac')
sluw_res = pipeline_phylogenetic(soil_table,
                                 rooted_tree_fixed,
                                 soil_mf,
                                 sampling_depth = 500,
                                 umap_metric = 'weighted_unifrac')

for out_, loc_, type_ in zip([kbue_res, kbur_res, kbua_res, kbub_res, kbuj_res, kbuu_res, kbuw_res, 
                              slue_res, slur_res, slua_res, slub_res, sluj_res, sluu_res, sluw_res,
                             ],
                              ['keyboard', 'keyboard', 'keyboard', 'keyboard', 'keyboard', 'keyboard', 'keyboard',
                               'soils', 'soils', 'soils', 'soils', 'soils', 'soils', 'soils',
                              ],
                              ['euclidean-umap-', 'euclidean-umap-rarefy500-', 'aitchison-umap-',
                               'braycurtis-umap-', 'jaccard-umap-', 'unweighted-unifrac-umap-',
                               'weighted-unifrac-umap-',
                               'euclidean-umap-', 'euclidean-umap-rarefy500-', 'aitchison-umap-',
                               'braycurtis-umap-', 'jaccard-umap-', 'unweighted-unifrac-umap-',
                               'weighted-unifrac-umap-',]):
    for name_, art_ in out_.__dict__.items():
        if name_ != '_fields':
            art_.save(os.path.join('results', loc_, type_ + name_))


NameError: name 'kb_table' is not defined

In [9]:
kbue_res = pipeline(kb_table,
                    kb_mf,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'euclidean')
kbur_res = pipeline(kb_table,
                    kb_mf,
                    sampling_depth = 500,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'euclidean')
kbua_res = pipeline(kb_table,
                    kb_mf,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'aitchison')
kbub_res = pipeline(kb_table,
                    kb_mf,
                    sampling_depth = 500,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'braycurtis')
kbuj_res = pipeline(kb_table,
                    kb_mf,
                    sampling_depth = 500,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'jaccard')
kbuu_res = pipeline_phylogenetic(kb_table,
                                 rooted_tree_fixed,
                                 kb_mf,
                                 sampling_depth = 500,
                                 umap_args = "{'n_neighbors': 80}",
                                 umap_metric = 'unweighted_unifrac')
kbuw_res = pipeline_phylogenetic(kb_table,
                                 rooted_tree_fixed,
                                 kb_mf,
                                 sampling_depth = 500,
                                 umap_args = "{'n_neighbors': 80}",
                                 umap_metric = 'weighted_unifrac')
slue_res = pipeline(soil_table,
                    soil_mf,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'euclidean')
slur_res = pipeline(soil_table,
                    soil_mf,
                    sampling_depth = 500,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'euclidean')
slua_res = pipeline(soil_table,
                    soil_mf,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'aitchison')
slub_res = pipeline(soil_table,
                    soil_mf,
                    sampling_depth = 500,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'braycurtis')
sluj_res = pipeline(soil_table,
                    soil_mf,
                    sampling_depth = 500,
                    umap_args = "{'n_neighbors': 80}",
                    umap_metric = 'jaccard')
sluu_res = pipeline_phylogenetic(soil_table,
                                 rooted_tree_fixed,
                                 soil_mf,
                                 sampling_depth = 500,
                                 umap_args = "{'n_neighbors': 80}",
                                 umap_metric = 'unweighted_unifrac')
sluw_res = pipeline_phylogenetic(soil_table,
                                 rooted_tree_fixed,
                                 soil_mf,
                                 sampling_depth = 500,
                                 umap_args = "{'n_neighbors': 80}",
                                 umap_metric = 'weighted_unifrac')

for out_, loc_, type_ in zip([kbue_res, kbur_res, kbua_res, kbub_res, kbuj_res, kbuu_res, kbuw_res, 
                              slue_res, slur_res, slua_res, slub_res, sluj_res, sluu_res, sluw_res,
                             ],
                              ['keyboard', 'keyboard', 'keyboard', 'keyboard', 'keyboard', 'keyboard', 'keyboard',
                               'soils', 'soils', 'soils', 'soils', 'soils', 'soils', 'soils',
                              ],
                              ['euclidean-umap-nn50-', 'euclidean-umap-rarefy500-nn50-', 'aitchison-umap-nn50-',
                               'braycurtis-umap-nn50-', 'jaccard-umap-nn50-', 'unweighted-unifrac-umap-nn50-',
                               'weighted-unifrac-umap-nn50-',
                               'euclidean-umap-nn50-', 'euclidean-umap-rarefy500-nn50-', 'aitchison-umap-nn50-',
                               'braycurtis-umap-nn50-', 'jaccard-umap-nn50-', 'unweighted-unifrac-umap-nn50-',
                               'weighted-unifrac-umap-nn50-',]):
    for name_, art_ in out_.__dict__.items():
        if name_ != '_fields':
            art_.save(os.path.join('results', loc_, type_ + name_))

/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [5]:
from qiime2.plugins.feature_table.actions import rarefy
from qiime2.plugins.diversity.actions import beta, pcoa
from qiime2.plugins.emperor.actions import plot as emperor_plot

In [6]:
# aitchison pcoa
loc_ = 'keyboard'
type_ = 'aitchison-mds-'
kb_dm = beta(kb_table, metric='aitchison')
kb_pcoa = pcoa(kb_dm.distance_matrix)
kb_emp = emperor_plot(kb_pcoa.pcoa, kb_mf)
out_ = {'distance_matrix': kb_dm.distance_matrix,
       'pcoa_results': kb_pcoa.pcoa,
       'emperor': kb_emp.visualization}
for name_, art_ in out_.items():
    if name_ != '_fields':
        art_.save(os.path.join('results', loc_, type_ + name_))

loc_ = 'soils'
type_ = 'aitchison-mds-'
sl_dm = beta(soil_table, metric='aitchison')
sl_pcoa = pcoa(sl_dm.distance_matrix)
sl_emp = emperor_plot(sl_pcoa.pcoa, soil_mf)
out_ = {'distance_matrix': sl_dm.distance_matrix,
       'pcoa_results': sl_pcoa.pcoa,
       'emperor': sl_emp.visualization}
for name_, art_ in out_.items():
    if name_ != '_fields':
        art_.save(os.path.join('results', loc_, type_ + name_))
        
loc_ = 'keyboard'
type_ = 'euclidean-mds-'
kb_dm = beta(kb_table, metric='euclidean')
kb_pcoa = pcoa(kb_dm.distance_matrix)
kb_emp = emperor_plot(kb_pcoa.pcoa, kb_mf)
out_ = {'distance_matrix': kb_dm.distance_matrix,
       'pcoa_results': kb_pcoa.pcoa,
       'emperor': kb_emp.visualization}
for name_, art_ in out_.items():
    if name_ != '_fields':
        art_.save(os.path.join('results', loc_, type_ + name_))

loc_ = 'soils'
type_ = 'euclidean-mds-'
sl_dm = beta(soil_table, metric='euclidean')
sl_pcoa = pcoa(sl_dm.distance_matrix)
sl_emp = emperor_plot(sl_pcoa.pcoa, soil_mf)
out_ = {'distance_matrix': sl_dm.distance_matrix,
       'pcoa_results': sl_pcoa.pcoa,
       'emperor': sl_emp.visualization}
for name_, art_ in out_.items():
    if name_ != '_fields':
        art_.save(os.path.join('results', loc_, type_ + name_))

In [7]:
kb_core = core_metrics_phylogenetic(kb_table,
                                    rooted_tree_fixed,
                                    500,
                                    kb_mf)
sl_core = core_metrics_phylogenetic(soil_table,
                                    rooted_tree_fixed,
                                    500,
                                    soil_mf)

for out_, loc_ in zip([kb_core, sl_core],
                      ['keyboard', 'soils']):
    for name_, art_ in out_.__dict__.items():
        if name_ != '_fields':
            art_.save(os.path.join('results', loc_, name_))


/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.08418897673658852 and the largest is 11.150986634883244.
  RuntimeWarning
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. 